# Lab 2: Create a Simple AI Agent

In this lab, we'll introduce you to AI agents by creating a simple agent that will create a bar graph based on data that we give to it. 

#### Step 1: Load packages

In [98]:
import os
from typing import Any
from pathlib import Path
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

load_dotenv() # Load environment variables from .env file

True

#### Step 2: Connect to your Azure AI Foundry project

In [99]:
# Connecting to our Azure AI Foundry project, which will allow us to use the deployed gpt-4o model
endpoint = os.environ.get("AIPROJECT_ENDPOINT", "https://ai-foundry-demo1.services.ai.azure.com/api/projects/firstProject")
model = os.getenv("CHAT_MODEL")

project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

#### Step 3: Create the simple AI Agent

In [100]:
with project_client:
    # Create an instance of the CodeInterpreterTool, which is responsible for generating the bar chart
    code_interpreter = CodeInterpreterTool()

    # The CodeInterpreterTool needs to be included in creation of the agent so that it can be used
    agent = project_client.agents.create_agent(
        model=model,
        name="my-agent-barchart",
        instructions="You are a helpful agent that creates data visualizations. Always save generated charts as files.",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread which is a conversation session between an agent and a user.
    thread = project_client.agents.threads.create()
    print(f"Created thread, thread ID: {thread.id}")

    # Create a prompt which contains the data + details for how the agent should generate the bar chart
    prompt = """Create a bar chart visualization using the following health insurance data:

Provider          Monthly Premium    Deductible    Out-of-Pocket Limit
Northwind         $300              $1,500        $6,000
Aetna             $350              $1,000        $5,500
United Health     $250              $2,000        $7,000
Premera           $200              $2,200        $6,500

Please:
1. Create a bar chart comparing the Monthly Premium for each provider
2. Save the chart as a PNG file
3. Make sure the chart is clear and properly labeled"""
    
    # Create a message, with the prompt being the message content that is sent to the model
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt,
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent to process the message in the thread
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Get all messages from the thread
    messages = project_client.agents.messages.list(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Check if the agent generated any files and download them
    file_found = False
    for msg in messages:
        print(f"Checking message from {msg.role}...")
        
        # Print the message content to see what the agent created
        if hasattr(msg, 'content') and msg.content:
            for content_item in msg.content:
                if hasattr(content_item, 'text') and content_item.text:
                    print(f"Agent response: {content_item.text.value}")
                    
                    # Check for file path annotations in the content
                    if hasattr(content_item.text, 'annotations'):
                        for annotation in content_item.text.annotations:
                            if hasattr(annotation, 'file_path') and hasattr(annotation.file_path, 'file_id'):
                                file_found = True
                                file_id = annotation.file_path.file_id
                                print(f"✅ Chart file created successfully!")
                                print(f"   File ID: {file_id}")
                                
                                # Extract filename from annotation text or use default
                                if hasattr(annotation, 'text') and annotation.text:
                                    file_name = Path(annotation.text).name
                                else:
                                    file_name = f"chart_{file_id}.png"
                                
                                print(f"   File name: {file_name}")
                                
                                # Try to download the file using the correct SDK methods
                                try:
                                    # Use the available get_content method from the agents files API
                                    file_content_generator = project_client.agents.files.get_content(file_id=file_id)
                                    
                                    # Convert generator to bytes
                                    file_content = b''.join(file_content_generator)
                                    
                                    # Save to current directory
                                    current_dir = Path.cwd()
                                    file_path = current_dir / file_name
                                    
                                    with open(file_path, 'wb') as f:
                                        f.write(file_content)
                                    
                                    print(f"📁 File successfully saved to: {file_path}")
                                    
                                except Exception as e:
                                    print(f"⚠️  Could not download using get_content: {e}")
                                    
                                    # Try alternative method using save
                                    try:
                                        current_dir = Path.cwd()
                                        file_path = current_dir / file_name
                                        
                                        # Use the save method
                                        project_client.agents.files.save(file_id=file_id, file_name=str(file_path))
                                        print(f"📁 File successfully saved using save method to: {file_path}")
                                        
                                    except Exception as save_e:
                                        print(f"⚠️  Could not download using save method: {save_e}")
                                        
                                        # Create a placeholder file with file info
                                        file_info = f"""Chart File Information:
File ID: {file_id}
File Name: {file_name}
Status: Generated by AI Agent
Available Methods: {[method for method in dir(project_client.agents.files) if not method.startswith('_')]}

The chart was successfully created! To access this file:
1. The file exists in the Azure AI environment with ID: {file_id}
2. You can try accessing it through the Azure AI Studio interface
3. Or use the file ID with other Azure AI SDK methods
"""
                                        info_file_path = Path.cwd() / f"chart_info_{file_id}.txt"
                                        with open(info_file_path, 'w') as f:
                                            f.write(file_info)
                                        
                                        print(f"📝 Created info file: {info_file_path}")

    if file_found:
        print("\n🎉 SUCCESS: The AI agent successfully created a bar chart visualization!")
        print("Files have been saved to the current directory.")
    else:
        print("\n⚠️  No chart files were detected in the agent's response.")
        print("The agent may have created content but not saved it as a file.")

    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Created agent, agent ID: asst_tBQMPsV2kY2SDWAV9cyNdUG7
Created thread, thread ID: thread_ZsiSPN991umGYnC8iJbfxJcL
Created thread, thread ID: thread_ZsiSPN991umGYnC8iJbfxJcL
Created message, message ID: msg_SLTZ9fALbhoMLyLeCj7BuTi0
Created message, message ID: msg_SLTZ9fALbhoMLyLeCj7BuTi0
Run finished with status: RunStatus.COMPLETED
Messages: <iterator object azure.core.paging.ItemPaged at 0x71e51537cc20>
Run finished with status: RunStatus.COMPLETED
Messages: <iterator object azure.core.paging.ItemPaged at 0x71e51537cc20>
Checking message from MessageRole.AGENT...
Agent response: The bar chart comparing the Monthly Premiums for each health insurance provider has been created and saved as a PNG file. You can download it using the link below:

[Download the chart](sandbox:/mnt/data/health_insurance_premiums.png)
✅ Chart file created successfully!
   File ID: assistant-4TZTtDvtfCR57av3dNQ1PU
   File name: health_insurance_premiums.png
Checking message from MessageRole.AGENT...
Agent resp